In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer,  text_to_word_sequence
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback, ModelCheckpoint
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed, Dropout
from keras import backend as K
from keras import optimizers
from keras.models import Model
import nltk
import re
import matplotlib.pyplot as plt
import sys
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from nltk import tokenize
import seaborn as sns

/home/evandro/myenv2/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/evandro/myenv2/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/evandro/myenv2/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/evandro/myenv2/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
class AttentionLayer(Layer):
    """
    Hierarchial Attention Layer as described by Hierarchical Attention Networks for Document Classification(2016)
    - Yang et. al.
    Source: https://www.cs.cmu.edu/~hovy/papers/16HLT-hierarchical-attention-networks.pdf
    Theano backend
    """
    def __init__(self,attention_dim=100,return_coefficients=False,**kwargs):
        # Initializer 
        self.supports_masking = True
        self.return_coefficients = return_coefficients
        self.init = initializers.get('glorot_uniform') # initializes values with uniform distribution
        self.attention_dim = attention_dim
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Builds all weights
        # W = Weight matrix, b = bias vector, u = context vector
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)),name='W')
        self.b = K.variable(self.init((self.attention_dim, )),name='b')
        self.u = K.variable(self.init((self.attention_dim, 1)),name='u')
        self.trainable_weights = [self.W, self.b, self.u]

        super(AttentionLayer, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, hit, mask=None):
        # Here, the actual calculation is done
        uit = K.bias_add(K.dot(hit, self.W),self.b)
        uit = K.tanh(uit)
        
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)
        ait = K.exp(ait)
        
        if mask is not None:
            ait *= K.cast(mask, K.floatx())

        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = hit * ait
        
        if self.return_coefficients:
            return [K.sum(weighted_input, axis=1), ait]
        else:
            return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        if self.return_coefficients:
            return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[-1], 1)]
        else:
            return input_shape[0], input_shape[-1]

    

In [3]:
MAX_SENTENCE_NUM = 95
MAX_WORD_NUM = 55
MAX_FEATURES = 200000 
EMBED_SIZE = 100

In [4]:
df=pd.read_csv("fakebr.csv",header=None)

In [5]:
df.columns = ['text', 'category']

In [6]:
df = df[['text', 'category']]

In [7]:
categories = df['category']
text = df['text']

In [8]:
import re
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub("\\n", " ", string)
    string = re.sub('\+\xa0', " ", string)
    string = re.sub("\xa0", " ", string)
    #string = re.sub(r"\u", " ", string)
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)  

    return string.strip().lower()

In [9]:
paras = []
labels = []
texts = []

In [10]:
labels = pd.get_dummies(categories)

In [11]:
sent_lens = []
sent_nums = []
for idx in range(df.text.shape[0]):
    text = clean_str(df.text[idx])
    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    sent_nums.append(len(sentences))
    for sent in sentences:
        sent_lens.append(len(text_to_word_sequence(sent)))
    paras.append(sentences)

In [12]:
tokenizer = Tokenizer(num_words=MAX_FEATURES, oov_token=True)
tokenizer.fit_on_texts(texts)

In [13]:
data = np.zeros((len(texts), MAX_SENTENCE_NUM, MAX_WORD_NUM), dtype='int64')
for i, sentences in enumerate(paras):
    for j, sent in enumerate(sentences):
        if j< MAX_SENTENCE_NUM:
            wordTokens = text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                try:
                    if k<MAX_WORD_NUM and tokenizer.word_index[word]<MAX_FEATURES:
                        data[i,j,k] = tokenizer.word_index[word]
                        k=k+1
                except:
                    print(word)
                    pass

In [14]:
import numpy as np
print(np.mean(sent_lens),2*np.std(sent_lens))
print(np.mean(sent_nums),2*np.std(sent_nums))

21.197674524355197 29.838632511480565
30.543333333333333 67.88308773177603


In [15]:
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

Total 98358 unique tokens.


In [16]:
print('Shape of data tensor:', data.shape)
print('Shape of labels tensor:', labels.shape)

Shape of data tensor: (7200, 95, 55)
Shape of labels tensor: (7200, 2)


In [17]:
GLOVE_DIR = "../embeddings/glove_s100.txt"
embeddings_index = {}
f = open(GLOVE_DIR)
for line in f:
    try:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except:
        print(word)
        pass
f.close()
print('Total %s word vectors.' % len(embeddings_index))

r$
00
三藏法師玄奘奉
r$
Total 929594 word vectors.


In [18]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBED_SIZE))
absent_words = 0
for word, i in word_index.items():
        
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None and len(embedding_vector) == 100:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        absent_words += 1
print('Total absent words are', absent_words, 'which is', "%0.2f" % (absent_words * 100 / len(word_index)), '% of total words')

Total absent words are 21097 which is 21.45 % of total words


In [19]:
def han():
    """
    Create Keras functional model for hierarchical attention network
    """
    embedding_layer = Embedding(len(word_index) + 1,EMBED_SIZE,weights=[embedding_matrix], input_length=MAX_WORD_NUM, trainable=False,name='word_embedding')

    # Words level attention model
    word_input = Input(shape=(MAX_WORD_NUM,), dtype='int32',name='word_input')
    word_sequences = embedding_layer(word_input)
    word_gru = Bidirectional(GRU(50, return_sequences=True),name='word_gru')(word_sequences)
    word_dense = Dense(100, activation='relu', name='word_dense')(word_gru) 
    word_att,word_coeffs = AttentionLayer(EMBED_SIZE,True,name='word_attention')(word_dense)
    wordEncoder = Model(inputs = word_input,outputs = word_att)

    # Sentence level attention model
    sent_input = Input(shape=(MAX_SENTENCE_NUM,MAX_WORD_NUM), dtype='int32',name='sent_input')
    sent_encoder = TimeDistributed(wordEncoder,name='sent_linking')(sent_input)
    sent_gru = Bidirectional(GRU(50, return_sequences=True),name='sent_gru')(sent_encoder)
    sent_dense = Dense(100, activation='relu', name='sent_dense')(sent_gru) 
    sent_att,sent_coeffs = AttentionLayer(EMBED_SIZE,return_coefficients=True,name='sent_attention')(sent_dense)
    sent_drop = Dropout(0.5,name='sent_dropout')(sent_att)
    preds = Dense(2, activation='softmax',name='output')(sent_drop)

    # Model compile
    model = Model(sent_input, preds)
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['categorical_accuracy'])
    print(wordEncoder.summary())
    print(model.summary())
    
    return model

In [20]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state= 0)

es = EarlyStopping(monitor = 'loss', min_delta = 1e-3, patience = 10) # Monitora a funcao de loss e se em 10 epocas ela nao melhorar em pelo menos 1e-3 o treinamento termina
rlr = ReduceLROnPlateau(monitor = 'loss', factor = 0.2, patience = 5) # Diminui a taxa de aprendizagem se ficar 5 epocas sem melhorar o loss
mcp = ModelCheckpoint(filepath = 'pesos.h5', monitor = 'loss', save_best_only = True, verbose = 1) # Salva o melhor modelo que encontrar

classificador = KerasClassifier(build_fn = han, epochs = 2, batch_size = 200)



In [21]:
#from sklearn.model_selection import cross_val_score


#resultados = cross_val_score(estimator = classificador,
#                             X = data, y = categories.values,
#                             cv =5, scoring = 'accuracy', 
#                             fit_params= {'callbacks': [es, rlr]}
#                            )

In [22]:
from keras.utils import to_categorical
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
labels_dummy = to_categorical(categories)

In [23]:
folds = list(skf.split(data, categories))
checkpoint = ModelCheckpoint('best_model.h5', verbose=0, monitor='val_loss',save_best_only=True, mode='auto')

In [24]:
res = []
acc = []
model = han()
model.save_weights('model.h5')
for j, (train_idx, val_idx) in enumerate(folds):
    
    model.load_weights('model.h5')
    
    print('\nFold ',j)
    X_train_cv = data[train_idx]
    y_train_cv = labels_dummy[train_idx]
    X_valid_cv = data[val_idx]
    y_valid_cv = labels_dummy[val_idx]
    
    history = model.fit(X_train_cv, y_train_cv, validation_data=(X_valid_cv, y_valid_cv), 
                        epochs = 100 , batch_size=400, callbacks=[checkpoint,es])
    
    y_pred = model.predict(X_valid_cv)
    
    res.append(precision_recall_fscore_support(y_valid_cv.argmax(axis=1), y_pred.argmax(axis=1)))
    acc.append(accuracy_score(y_valid_cv.argmax(axis=1), y_pred.argmax(axis=1)))
    
 






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_input (InputLayer)      (None, 55)                0         
_________________________________________________________________
word_embedding (Embedding)   (None, 55, 100)           9835900   
_________________________________________________________________
word_gru (Bidirectional)     (None, 55, 100)           45300     
_________________________________________________________________
word_dense (Dense)           (None, 55, 100)           10100     
_________________________________________________________________
word_attention (AttentionLay [(None, 100), (None, 100, 10200     
Total params: 9,901,500
Trainable params: 65,600
Non-trainable params: 9,835,900
_______________________________________________________

Epoch 24/100
5760/5760 [==============================] - 101s 18ms/step - loss: 0.0692 - categorical_accuracy: 0.9767 - val_loss: 0.1039 - val_categorical_accuracy: 0.9632
Epoch 25/100
5760/5760 [==============================] - 101s 18ms/step - loss: 0.0690 - categorical_accuracy: 0.9764 - val_loss: 0.1016 - val_categorical_accuracy: 0.9646
Epoch 26/100
5760/5760 [==============================] - 101s 18ms/step - loss: 0.0514 - categorical_accuracy: 0.9832 - val_loss: 0.1054 - val_categorical_accuracy: 0.9674
Epoch 27/100
5760/5760 [==============================] - 101s 18ms/step - loss: 0.0391 - categorical_accuracy: 0.9865 - val_loss: 0.1163 - val_categorical_accuracy: 0.9660
Epoch 28/100
5760/5760 [==============================] - 101s 18ms/step - loss: 0.0296 - categorical_accuracy: 0.9910 - val_loss: 0.1325 - val_categorical_accuracy: 0.9639
Epoch 29/100
5760/5760 [==============================] - 102s 18ms/step - loss: 0.0197 - categorical_accuracy: 0.9944 - val_loss: 0.16

5760/5760 [==============================] - 96s 17ms/step - loss: 0.1513 - categorical_accuracy: 0.9470 - val_loss: 0.1652 - val_categorical_accuracy: 0.9403
Epoch 7/100
5760/5760 [==============================] - 96s 17ms/step - loss: 0.1433 - categorical_accuracy: 0.9521 - val_loss: 0.1439 - val_categorical_accuracy: 0.9521
Epoch 8/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.1375 - categorical_accuracy: 0.9531 - val_loss: 0.1434 - val_categorical_accuracy: 0.9458
Epoch 9/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.1187 - categorical_accuracy: 0.9575 - val_loss: 0.1253 - val_categorical_accuracy: 0.9535
Epoch 10/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.1142 - categorical_accuracy: 0.9613 - val_loss: 0.1219 - val_categorical_accuracy: 0.9556
Epoch 11/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.1102 - categorical_accuracy: 0.9606 - val_loss: 0.1330 - val_categorical_a

Epoch 54/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.0114 - categorical_accuracy: 0.9965 - val_loss: 0.1666 - val_categorical_accuracy: 0.9660
Epoch 55/100
5760/5760 [==============================] - 96s 17ms/step - loss: 0.0050 - categorical_accuracy: 0.9986 - val_loss: 0.2018 - val_categorical_accuracy: 0.9604
Epoch 56/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.0042 - categorical_accuracy: 0.9983 - val_loss: 0.1825 - val_categorical_accuracy: 0.9660
Epoch 57/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.0015 - categorical_accuracy: 0.9998 - val_loss: 0.1956 - val_categorical_accuracy: 0.9618
Epoch 58/100
5760/5760 [==============================] - 96s 17ms/step - loss: 0.0013 - categorical_accuracy: 0.9997 - val_loss: 0.2074 - val_categorical_accuracy: 0.9632
Epoch 59/100
5760/5760 [==============================] - 97s 17ms/step - loss: 9.2624e-04 - categorical_accuracy: 0.9998 - val_loss: 0.2240

5760/5760 [==============================] - 97s 17ms/step - loss: 0.0646 - categorical_accuracy: 0.9792 - val_loss: 0.0953 - val_categorical_accuracy: 0.9646
Epoch 35/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.0593 - categorical_accuracy: 0.9816 - val_loss: 0.0849 - val_categorical_accuracy: 0.9660
Epoch 36/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.0560 - categorical_accuracy: 0.9828 - val_loss: 0.0822 - val_categorical_accuracy: 0.9681
Epoch 37/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.0506 - categorical_accuracy: 0.9828 - val_loss: 0.0909 - val_categorical_accuracy: 0.9653
Epoch 38/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.0509 - categorical_accuracy: 0.9826 - val_loss: 0.0873 - val_categorical_accuracy: 0.9667
Epoch 39/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.0423 - categorical_accuracy: 0.9877 - val_loss: 0.0935 - val_categorica

Epoch 83/100
5760/5760 [==============================] - 96s 17ms/step - loss: 0.0033 - categorical_accuracy: 0.9995 - val_loss: 0.1792 - val_categorical_accuracy: 0.9715
Epoch 84/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.0029 - categorical_accuracy: 0.9995 - val_loss: 0.1903 - val_categorical_accuracy: 0.9701
Epoch 85/100
5760/5760 [==============================] - 96s 17ms/step - loss: 0.0034 - categorical_accuracy: 0.9993 - val_loss: 0.1898 - val_categorical_accuracy: 0.9708
Epoch 86/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.0282 - categorical_accuracy: 0.9927 - val_loss: 0.6589 - val_categorical_accuracy: 0.9007
Epoch 87/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.2239 - categorical_accuracy: 0.9491 - val_loss: 0.1618 - val_categorical_accuracy: 0.9542
Epoch 88/100
5760/5760 [==============================] - 98s 17ms/step - loss: 0.0804 - categorical_accuracy: 0.9773 - val_loss: 0.0901 - v

5760/5760 [==============================] - 96s 17ms/step - loss: 0.0517 - categorical_accuracy: 0.9845 - val_loss: 0.0717 - val_categorical_accuracy: 0.9785
Epoch 40/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.0437 - categorical_accuracy: 0.9865 - val_loss: 0.0721 - val_categorical_accuracy: 0.9743
Epoch 41/100
5760/5760 [==============================] - 96s 17ms/step - loss: 0.0364 - categorical_accuracy: 0.9894 - val_loss: 0.0801 - val_categorical_accuracy: 0.9771
Epoch 42/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.0353 - categorical_accuracy: 0.9917 - val_loss: 0.0816 - val_categorical_accuracy: 0.9736
Epoch 43/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.0307 - categorical_accuracy: 0.9936 - val_loss: 0.0763 - val_categorical_accuracy: 0.9764
Epoch 44/100
5760/5760 [==============================] - 96s 17ms/step - loss: 0.0282 - categorical_accuracy: 0.9911 - val_loss: 0.0783 - val_categorica

5760/5760 [==============================] - 96s 17ms/step - loss: 0.1822 - categorical_accuracy: 0.9359 - val_loss: 0.1397 - val_categorical_accuracy: 0.9507
Epoch 7/100
5760/5760 [==============================] - 96s 17ms/step - loss: 0.1691 - categorical_accuracy: 0.9427 - val_loss: 0.1344 - val_categorical_accuracy: 0.9542
Epoch 8/100
5760/5760 [==============================] - 96s 17ms/step - loss: 0.1613 - categorical_accuracy: 0.9437 - val_loss: 0.1302 - val_categorical_accuracy: 0.9549
Epoch 9/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.1567 - categorical_accuracy: 0.9472 - val_loss: 0.1263 - val_categorical_accuracy: 0.9563
Epoch 10/100
5760/5760 [==============================] - 96s 17ms/step - loss: 0.1557 - categorical_accuracy: 0.9469 - val_loss: 0.1151 - val_categorical_accuracy: 0.9583
Epoch 11/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.1500 - categorical_accuracy: 0.9490 - val_loss: 0.1475 - val_categorical_a

Epoch 54/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.0176 - categorical_accuracy: 0.9946 - val_loss: 0.1355 - val_categorical_accuracy: 0.9701
Epoch 55/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.0233 - categorical_accuracy: 0.9932 - val_loss: 0.1494 - val_categorical_accuracy: 0.9667
Epoch 56/100
5760/5760 [==============================] - 96s 17ms/step - loss: 0.0164 - categorical_accuracy: 0.9958 - val_loss: 0.1327 - val_categorical_accuracy: 0.9729
Epoch 57/100
5760/5760 [==============================] - 96s 17ms/step - loss: 0.0110 - categorical_accuracy: 0.9976 - val_loss: 0.1459 - val_categorical_accuracy: 0.9701
Epoch 58/100
5760/5760 [==============================] - 97s 17ms/step - loss: 0.0103 - categorical_accuracy: 0.9981 - val_loss: 0.1634 - val_categorical_accuracy: 0.9674
Epoch 59/100
5760/5760 [==============================] - 96s 17ms/step - loss: 0.0088 - categorical_accuracy: 0.9988 - val_loss: 0.1725 - v

In [25]:
r = model.predict(X_valid_cv)

In [26]:
categories.values

array([0, 0, 0, ..., 1, 1, 1])

In [27]:
type(labels_dummy)

numpy.ndarray

In [28]:
resultados

NameError: name 'resultados' is not defined

In [ ]:
for i in r.argmax(axis=1):
    print(i)

In [29]:
res

[(array([0.97075209, 0.96814404]),
  array([0.96805556, 0.97083333]),
  array([0.96940195, 0.96948682]),
  array([720, 720])),
 (array([0.95225102, 0.9688826 ]),
  array([0.96944444, 0.95138889]),
  array([0.96077082, 0.96005606]),
  array([720, 720])),
 (array([0.98041958, 0.9737931 ]),
  array([0.97361111, 0.98055556]),
  array([0.97700348, 0.97716263]),
  array([720, 720])),
 (array([0.9762901 , 0.97233748]),
  array([0.97222222, 0.97638889]),
  array([0.97425191, 0.97435897]),
  array([720, 720])),
 (array([0.96027397, 0.97323944]),
  array([0.97361111, 0.95972222]),
  array([0.96689655, 0.96643357]),
  array([720, 720]))]

In [30]:
resultados = pd.DataFrame(columns=['Precision', 'PrecisionT', 'Recall', 'RecallT', 'Fscore', 'FscoreT', 'Acc'])
for i in range(5):
    resultados = resultados.append({'Precision':res[i][0][0], 'PrecisionT':res[i][0][1], 
                                    'Recall':res[i][1][0], 'RecallT':res[i][1][1], 'Fscore':res[i][2][0], 
                                    'FscoreT':res[i][2][1], 'Acc':acc[i]},ignore_index=True)
    print(resultados)

   Precision  PrecisionT    Recall   RecallT    Fscore   FscoreT       Acc
0   0.970752    0.968144  0.968056  0.970833  0.969402  0.969487  0.969444
   Precision  PrecisionT    Recall   RecallT    Fscore   FscoreT       Acc
0   0.970752    0.968144  0.968056  0.970833  0.969402  0.969487  0.969444
1   0.952251    0.968883  0.969444  0.951389  0.960771  0.960056  0.960417
   Precision  PrecisionT    Recall   RecallT    Fscore   FscoreT       Acc
0   0.970752    0.968144  0.968056  0.970833  0.969402  0.969487  0.969444
1   0.952251    0.968883  0.969444  0.951389  0.960771  0.960056  0.960417
2   0.980420    0.973793  0.973611  0.980556  0.977003  0.977163  0.977083
   Precision  PrecisionT    Recall   RecallT    Fscore   FscoreT       Acc
0   0.970752    0.968144  0.968056  0.970833  0.969402  0.969487  0.969444
1   0.952251    0.968883  0.969444  0.951389  0.960771  0.960056  0.960417
2   0.980420    0.973793  0.973611  0.980556  0.977003  0.977163  0.977083
3   0.976290    0.972337 

In [31]:
resultados.to_csv("res.csv")